In [ ]:
from dewloosh.core.abc import ABCMeta_Weak
from dewloosh.core.signature import Signature

from copy import deepcopy
from functools import partial
from typing import Callable, Any

In [ ]:
def _update_func(func1, func2 : Callable):
    """
    Updates self with the content of other. Both must be abstracts.
    """
    return func1.signature.update_func(func2.signature)


def _accepts_func(func1, func2 : Callable) -> bool:
    """
    Returns True if other (implemented operation) is compatible to
    self (abstract operation). Every value to every key of self must be
    present in other.
    """
    return func1.signature.accepts_func(func2.signature)


def operation(*args):

    def decorator(funcobj):
        setattr(funcobj, 'isoperation', True)
        setattr(funcobj, 'isabstractoperation', False)
        setattr(funcobj, 'signature', Signature.from_func(funcobj, *args))
        setattr(funcobj, 'update', partial(_update_func, funcobj))
        setattr(funcobj, 'accepts', partial(_accepts_func, funcobj))
        return funcobj

    return decorator


def abstractoperation(*args):

    def decorator(funcobj):
        setattr(funcobj, 'isoperation', True)
        setattr(funcobj, 'isabstractoperation', True)
        setattr(funcobj, 'signature', Signature.from_func(funcobj,*args))
        setattr(funcobj, 'update', partial(_update_func, funcobj))
        setattr(funcobj, 'accepts', partial(_accepts_func, funcobj))
        return funcobj

    return decorator


def abstractattribute(funcobj):
    setattr(funcobj, 'isattribute', True)
    setattr(funcobj, 'isabstractattribute', True)
    setattr(funcobj, 'signature', Signature.from_property(funcobj))
    return funcobj


def attribute(funcobj):
    return property(funcobj)

In [ ]:
class ABCMeta_Algebra(ABCMeta_Weak):
    """
    Metaclass for defining ABCs for algebraic structures.
    """

    def __new__(metaclass, name, bases, namespace, *args, **kwargs):
        """
        This solution breakes the behaviour of ABCMeta, probably
        because set is a class instance, not a type.

        # _bases = []
        # for base in bases:
        #     _bases.append(base)
        # _bases.append(set)
        # bases = tuple(_bases)

        Instead, the helper class Algebra is wrapped around a set.
        """
        namespace['abstractoperations'] = \
            metaclass._get_abstractoperations(bases, namespace)
        namespace['operations'] = \
            metaclass._get_operations(bases, namespace)
        namespace['abstractattributes'] = \
            metaclass._get_abstractattributes(bases, namespace)
        cls = super().__new__(metaclass, name, bases,
                              namespace, *args, **kwargs)
        return cls

    def _get_abstractoperations(bases, namespace) -> dict:
        """
        Returns a dict of required abstract operations. Abstracts with
        identical names are fused.
        """
        required = dict()
        for base in bases:
            d = getattr(base, "abstractoperations", dict())
            for name, sig in d.items():
                if name not in required:
                    required[name] = deepcopy(sig)
                else:
                    required[name].update(sig)

        for name, item in namespace.items():
            if callable(item) and getattr(item, 'isabstractoperation', False):
                if name not in required:
                    required[name] = deepcopy(item.signature)
                else:
                    required[name].update(item.signature)
        return required

    def _get_operations(bases, namespace) -> dict:
        """
        Returns a dict of implemented operations.
        """
        implemented = dict()
        for base in bases:
            d = getattr(base, "operations", dict())
            for name, sig in d.items():
                implemented[name] = deepcopy(sig)

        for name, item in namespace.items():
            if callable(item) and getattr(item, 'isoperation', False):
                if not item.isabstractoperation:
                    implemented[name] = deepcopy(item.signature)
        return implemented

    def _get_abstractattributes(bases, namespace) -> dict:
        """
        Returns a dict of required abstract attributes.
        """
        required = dict()
        for base in bases:
            d = getattr(base, "abstractattributes", dict())
            for name, sig in d.items():
                required[name] = deepcopy(sig)

        for name, item in namespace.items():
            if getattr(item, 'isabstractattribute', False):
                required[name] = deepcopy(item.signature)

        return required

    # def __instancecheck__(cls, inst):
    #     return cls.__signature__ == inst.signature


# This solution also works
# from pyoneer.decorators import wraparound
#@wraparound(set)
class Algebra_ABC(metaclass = ABCMeta_Algebra):
    """
    Helper class that provides a standard way to create an ABC_Algebra
    using inheritance.

    An algebraic structure on a set A (called the underlying set, carrier
    set or domain) is a collection of operations on A of finite arity,
    together with a finite set of identities, called axioms of the structure
    that these operations must satisfy.
    """
    __slots__ = ()

    def __init__(self, *args, **kwargs):
        super().__init__()
        self._check_attributes()

    def __new__(cls, *args, **kwargs):
        obj = super().__new__(cls)
        Algebra_ABC._check_operations(cls)
        return obj

    def _check_operations(cls):
        required = cls.abstractoperations
        implemented = cls.operations

        for key,sig in required.items():
            problem = False
            if key in implemented:
                problem = not sig.accepts_op(implemented[key])
            else:
                problem = True
            if problem:
                raise TypeError("""Class {} can't be initiated with \
                                abstract operation {}!""".format(cls,key))
        return None

    def _check_attributes(self):
        required = type(self).abstractattributes

        for key,sig in required.items():
            problem = False
            if hasattr(self, key):
                attr = getattr(self, key, None)
                problem = not sig.accepts_attr(attr)
            else:
                problem = True
            if problem:
                raise TypeError("""Object {} can't be initiated with \
                                abstract attribute {}!""".format(self,key))
        return None


#This solution also works
#@wraparound(set)
class Set(set):

    @property
    def cardinality(self):
        """
        Returns the number of the elements in the set.
        """
        return len(self)


class AlgebraicStructure(Algebra_ABC, Set):
    """
    An algebraic structure on a set A (called the underlying set, carrier set
    or domain) is a collection of operations on A of finite arity, together
    with a finite set of identities, called axioms of the structure that
    these operations must satisfy.
    """
    def __init__(self, *args, **kwargs):
        self.dtype = kwargs.pop('dtype', None)
        super().__init__(*args, **kwargs)
   

In [ ]:
from typing import Union
from decimal import Decimal

Scalar = Union[float, complex, Decimal]

class Structure(AlgebraicStructure):

    @abstractattribute
    def dimension(self) -> int: ...

    @abstractoperation('closure', 'associativity',
                        'neutrality', 'inversibility')
    def multiplication(self, a : Scalar, b : Scalar) -> Scalar:
        pass

class Group(AlgebraicStructure):

    def __init__(self, dim : int = 0):
        self.dim = dim
        super().__init__()

    @attribute
    def dimension(self):
        return self.dim

    @operation('closure', 'associativity', 'commutativity',
                'neutrality', 'inversibility')
    def addition(self,a,b) -> float:
        return a+b

    @operation('closure', 'associativity',
                'neutrality', 'inversibility')
    def multiplication(self, a : float, b : float) -> float:
        return a*b

print(Structure.multiplication.accepts(Group.multiplication))

group = Group()
group.add(1.0)
print(group.addition(1.0,2.0))
print(group.multiplication(1.0,2.0))
print(Group.operations)
print(Group.abstractoperations)

@property
def foo():
    return 2